## Import Libraries

In [35]:
import pandas as pd
import lonely_boy2 as lb2
from sklearn import preprocessing
import numpy as np
from tqdm import tqdm_notebook

## Initialize Relation Attributes

In [11]:
usr_ratings_cols = 'user id | item id | rating | timestamp'.split(' | ')
movie_info_cols = '''movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'''.split(' | ')
usr_info_cols = 'user id | age | gender | occupation | zip code'.split(' | ')

## Initialize Relation URL's

In [12]:
usr_ratings_url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
movie_info_url  = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.item'
usr_info_url    = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.user'
genre_info_url  = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.genre'
ocp_info_url    = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.occupation'

## Read CSV's

In [13]:
usr_ratings = pd.read_csv(usr_ratings_url, delimiter='\t', header=None, names=usr_ratings_cols, encoding='latin-1')

movie_info = pd.read_csv(movie_info_url, delimiter='|', header=None, names=movie_info_cols, encoding='latin-1')

usr_info = pd.read_csv(usr_info_url, delimiter='|', header=None, names=usr_info_cols, encoding='latin-1')

genre_info = pd.read_csv(genre_info_url, delimiter='|', header=None, names=['genre_id', 'genre_code'], encoding='latin-1')
genre_info = genre_info[['genre_id']]

ocp_info = pd.read_csv(ocp_info_url, delimiter='|', header=None, names=['ocp_id'], encoding='latin-1')
ocp_info = ocp_info[['ocp_id']]

In [14]:
usr_ratings.head(2)

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [15]:
movie_info.head(2)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [41]:
usr_info.head(10)

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [17]:
genre_info.head(2)

,genre_id
0,unknown
1,Action


In [50]:
ocp_info


,ocp_id
0,administrator
1,artist
2,doctor
3,educator
4,engineer
5,entertainment
6,executive
7,healthcare
8,homemaker
9,lawyer


In [28]:
ocp_info[ocp_id == 'administrator'].index

NameError: name 'ocp_id' is not defined

In [52]:
ocp_info.index[ocp_info['ocp_id'] == 'artist'][0]

1

In [44]:
usr_info['occupation_encoded']='abc'

In [53]:
for index,row in tqdm_notebook(usr_info['occupation'].iteritems()):
    usr_info['occupation_encoded'][index] = ocp_info.index[ocp_info['ocp_id'] == row][0]
    
    
    

A Jupyter Widget

C:\Users\theotheo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [56]:
bingend = list(map(lambda x:0 if (x=='M') else 1,usr_info['gender'].values))

In [57]:
col = pd.Series(bingend)
usr_info['bingender'] = col.values

In [61]:
usr_info = usr_info.drop(['gender','occupation','zip code'],axis=1)

In [64]:
usr_info.head()

,user id,age,occupation_encoded,bingender
0,1,24,19,0
1,2,53,13,1
2,3,23,20,0
3,4,24,19,0
4,5,33,13,1


In [39]:
pd.factorize(ocp_info.ocp_id)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20], dtype=int64),
 Index(['administrator', 'artist', 'doctor', 'educator', 'engineer',
        'entertainment', 'executive', 'healthcare', 'homemaker', 'lawyer',
        'librarian', 'marketing', 'none', 'other', 'programmer', 'retired',
        'salesman', 'scientist', 'student', 'technician', 'writer'],
       dtype='object'))

In [40]:
usr_info['occupation'][5]

'executive'

In [11]:
TheDataData = usr_info.iloc[:,:-1]

In [12]:
TheDataData = TheDataData.merge(usr_ratings, on='user id')

In [13]:
TheDataData = TheDataData.drop(['timestamp', 'user id'], axis=1)

In [14]:
TheDataData = TheDataData.merge(movie_info, left_on = 'item id', right_on = 'movie id').drop(['item id', 'movie id', 'movie title', 'release date', 'video release date', 'IMDb URL'], axis=1)

In [19]:
TheDataData

,age,gender,occupation,rating,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,24,M,technician,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47,M,educator,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35,F,other,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,27,M,programmer,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,49,M,educator,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,50,M,healthcare,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,20,M,student,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,26,M,student,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,53,M,programmer,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,25,M,student,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
TheDataData.groupby(['age','gender', 'occupation', 'rating'],as_index = False).sum()

,age,gender,occupation,rating,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,7,M,student,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,7,M,student,2,0,2,2,0,0,1,...,0,0,0,1,0,0,0,1,0,0
2,7,M,student,3,0,6,5,0,1,4,...,0,0,0,0,0,2,2,4,1,0
3,7,M,student,4,0,7,6,1,4,9,...,1,0,0,2,0,5,6,3,2,0
4,7,M,student,5,0,3,1,0,0,4,...,0,0,0,0,1,4,2,1,2,1
5,10,M,student,1,0,0,0,4,4,0,...,0,0,0,4,0,0,0,0,0,0
6,10,M,student,2,0,0,1,4,5,1,...,0,0,0,1,0,0,0,0,0,0
7,10,M,student,3,0,1,2,3,4,0,...,0,0,0,3,0,1,1,0,1,0
8,10,M,student,4,0,1,1,3,4,4,...,0,0,0,1,1,2,1,0,1,0
9,10,M,student,5,0,1,3,2,9,7,...,0,0,0,0,0,1,1,0,0,0


## Create one-hot Coded DataFrames